Connected to base (Python 3.11.5)

In [1]:
from datasets import load_dataset


dataset = load_dataset("cifar100").with_format("torch")

Found cached dataset parquet (/home/cesarruiz/.cache/huggingface/datasets/parquet/cifar100-c6fdc5677f3c66a8/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
subset = dataset["train"][0:20]

In [3]:
subset

{'img': tensor([[[[255, 255, 255],
           [255, 255, 255],
           [255, 255, 255],
           ...,
           [195, 205, 193],
           [212, 224, 204],
           [182, 194, 167]],
 
          [[255, 255, 255],
           [254, 254, 254],
           [254, 254, 254],
           ...,
           [170, 176, 150],
           [161, 168, 130],
           [146, 154, 113]],
 
          [[255, 255, 255],
           [254, 254, 254],
           [255, 255, 255],
           ...,
           [189, 199, 169],
           [166, 178, 130],
           [121, 133,  87]],
 
          ...,
 
          [[148, 185,  79],
           [142, 182,  57],
           [140, 179,  60],
           ...,
           [ 30,  17,   1],
           [ 65,  62,  15],
           [ 76,  77,  20]],
 
          [[122, 157,  66],
           [120, 155,  58],
           [126, 160,  71],
           ...,
           [ 22,  16,   3],
           [ 97, 112,  56],
           [141, 161,  87]],
 
          [[ 87, 122,  41],
           [ 

In [4]:
import numpy as np
import torch

In [5]:
ran1 = torch.tensor(np.random.choice(range(100), size=20))
ran2 = torch.tensor(np.random.choice([19], size=20))

In [6]:
q = len(subset["img"])

In [ ]:
from utils import plot_image_comparison
plot_image_comparison(subset["img"],subset["img"],ran1, ran2, subset["fine_label"], True, 4)